# spellchk

In [1]:
from default import *
from io import StringIO
import numpy as np
import Levenshtein

## Documentation

Read `answer/default.py` starting with the `spellchk` function and see how it solves the task of spell correction using a pre-trained language model that can predict a replacement token for a masked token in the input.

In your submission, write some beautiful documentation of your program here.

Our program uses a pre-trained language model that provides 1000 candidates to replace the typo word. Then, we employ Levenshtein distance (edit distance) between the typo and the candidates. This distance is in fact the minimum number of single-character edits (insertions, deletions or substitutions) required to change one word into the other. It basically measures how much a candidate is close to the typo word. We convert this distance to a number between 0 and 1, which resembles a probability number in order to be able to combine it with the candidates' score generated by the language model. These two probability numbers are combined together with a weight, and the candidate with the highest score will be selected.
Note that the utilized language model is uncased, and all the generated candidates are lower case. In the sentences however, some of the typo words need to be capital. Therefore, we capitalize the final selected candidate if the typo word is also capital. In the following, you can see the changes we made to the "spellchk" and "select_correction" functions.

In [2]:
def select_correction(typo, predict):
    # return the most likely prediction for the mask token
    LM_score = np.array([predict[i]['score'] for i in range(len(predict))])
    edit_dist = np.array([Levenshtein.distance(typo.lower(), predict[i]['token_str']) for i in range(len(predict))])
    edit_score = (edit_dist.max() - edit_dist) / edit_dist.max()
    w = 0.95
    score_tot = w * edit_score + (1-w) * LM_score
    best_ind = np.argmax(score_tot)
    return predict[best_ind]['token_str'].capitalize() if typo[0].isupper() else predict[best_ind]['token_str']

def spellchk(fh):
    for (locations, sent) in get_typo_locations(fh):
        spellchk_sent = sent
        for i in locations:
            # predict top_k replacements only for the typo word at index i
            predict = fill_mask(
                " ".join([ sent[j] if j != i else mask for j in range(len(sent)) ]), 
                top_k=1000
            )
            logging.info(predict)
            spellchk_sent[i] = select_correction(sent[i], predict)
        yield(locations, spellchk_sent)

## Analysis

In the default solution, "select_correction" function just select the first candidate of the language model with the highest score. In fact, this approach does not take the typo word into account at all. It is not surprising that its accuracy is as low as 0.23 on the dev dataset. Here are the default "spellchk" and "select_correction" functions:

In [3]:
def select_correction(typo, predict):
    # return the most likely prediction for the mask token
    return predict[0]['token_str']

def spellchk(fh):
    for (locations, sent) in get_typo_locations(fh):
        spellchk_sent = sent
        for i in locations:
            # predict top_k replacements only for the typo word at index i
            predict = fill_mask(
                " ".join([ sent[j] if j != i else mask for j in range(len(sent)) ]), 
                top_k=20
            )
            logging.info(predict)
            spellchk_sent[i] = select_correction(sent[i], predict)
        yield(locations, spellchk_sent)

In [4]:

with StringIO("4\tit will put your maind into non-stop learning.\n \
              3\t`` Sad '' wss not the right word , of course .\n \
              5,14	Just before Myra left -- Sue was saying good-by to Cathy , and she didm't realize I was near '' .") as f:
    for (locations, spellchk_sent) in spellchk(f):
        print("{locs}\t{sent}".format(
            locs=",".join([str(i) for i in locations]),
            sent=" ".join(spellchk_sent)
        ))

4	it will put your mind into non-stop learning.
3	`` Sad '' s not the right word , of course .
5,14	Just before Myra left -- cathy was saying good-by to Cathy , and she did realize I was near '' .


You can see that the program replaces "wss" with "s" and "Sue" with "cathy" which has nothing to do with the typo word.

So, in our first attempt, we counted the number of common words between the typo and the candidates using the set intersection and selected the one with highest number of common words. This approach improved the accuracy on the dev dataset to 0.46, but it is still low.

In [5]:
def select_correction(typo, predict):
    # return the most likely prediction for the mask token
    dist_score = [len(set(typo.lower()).intersection(set(predict[i]['token_str'])))/len(typo) for i in range(len(predict))]
    best_ind = np.argmax(dist_score)
    return predict[best_ind]['token_str']

def spellchk(fh):
    for (locations, sent) in get_typo_locations(fh):
        spellchk_sent = sent
        for i in locations:
            # predict top_k replacements only for the typo word at index i
            predict = fill_mask(
                " ".join([ sent[j] if j != i else mask for j in range(len(sent)) ]), 
                top_k=20
            )
            logging.info(predict)
            spellchk_sent[i] = select_correction(sent[i], predict)
        yield(locations, spellchk_sent)

In [6]:
with StringIO("4\tit will put your maind into non-stop learning.\n \
              3\t`` Sad '' wss not the right word , of course .\n \
              5,14	Just before Myra left -- Sue was saying good-by to Cathy , and she didm't realize I was near '' .") as f:
    for (locations, spellchk_sent) in spellchk(f):
        print("{locs}\t{sent}".format(
            locs=",".join([str(i) for i in locations]),
            sent=" ".join(spellchk_sent)
        ))

4	it will put your mind into non-stop learning.
3	`` Sad '' was not the right word , of course .
5,14	Just before Myra left -- susie was saying good-by to Cathy , and she immediately realize I was near '' .


Now, you can see that "was" is detected correctly but "Sue" is replaced with "susie" and "didm't" is replaced with "immediately"! Because all the characters of "Sue" and "didm't" are available in "susie" and "immediately". Therefore, set intersection might not be a good choice.

That's why we switched to a better distance metric between two strings which measures the number of single-character edits needed to change one word to another. It is called "Levenshtein" distance. In this case, the accuracy became 0.52.

In [7]:
def select_correction(typo, predict):
    # return the most likely prediction for the mask token
    edit_dist = [Levenshtein.distance(typo.lower(), predict[i]['token_str']) for i in range(len(predict))]
    best_ind = np.argmin(edit_dist)
    return predict[best_ind]['token_str']

def spellchk(fh):
    for (locations, sent) in get_typo_locations(fh):
        spellchk_sent = sent
        for i in locations:
            # predict top_k replacements only for the typo word at index i
            predict = fill_mask(
                " ".join([ sent[j] if j != i else mask for j in range(len(sent)) ]), 
                top_k=20
            )
            logging.info(predict)
            spellchk_sent[i] = select_correction(sent[i], predict)
        yield(locations, spellchk_sent)

In [8]:
with StringIO("4\tit will put your maind into non-stop learning.\n \
              3\t`` Sad '' wss not the right word , of course .\n \
              5,14	Just before Myra left -- Sue was saying good-by to Cathy , and she didm't realize I was near '' .") as f:
    for (locations, spellchk_sent) in spellchk(f):
        print("{locs}\t{sent}".format(
            locs=",".join([str(i) for i in locations]),
            sent=" ".join(spellchk_sent)
        ))

4	it will put your mind into non-stop learning.
3	`` Sad '' was not the right word , of course .
5,14	Just before Myra left -- she was saying good-by to Cathy , and she did realize I was near '' .


Now, "Sue" is correctly replaced with "she", but the problem is that "she" is not capital. The next change we made, was to capitalize the output words if the typo is capital. The other thing we did was to convert the Levenshtein distance to a probability-like number and combine it with the language model score. The latter didn't improve the accuracy. However, capitalization improved it to 0.57.

In [9]:
def select_correction(typo, predict):
    # return the most likely prediction for the mask token
    LM_score = np.array([predict[i]['score'] for i in range(len(predict))])
    edit_dist = np.array([Levenshtein.distance(typo.lower(), predict[i]['token_str']) for i in range(len(predict))])
    edit_score = (edit_dist.max() - edit_dist) / edit_dist.max()
    w = 0.95
    score_tot = w * edit_score + (1-w) * LM_score
    best_ind = np.argmax(score_tot)
    return predict[best_ind]['token_str'].capitalize() if typo[0].isupper() else predict[best_ind]['token_str']
def spellchk(fh):
    for (locations, sent) in get_typo_locations(fh):
        spellchk_sent = sent
        for i in locations:
            # predict top_k replacements only for the typo word at index i
            predict = fill_mask(
                " ".join([ sent[j] if j != i else mask for j in range(len(sent)) ]), 
                top_k=20
            )
            logging.info(predict)
            spellchk_sent[i] = select_correction(sent[i], predict)
        yield(locations, spellchk_sent)

In [10]:
with StringIO("4\tit will put your maind into non-stop learning.\n \
              3\t`` Sad '' wss not the right word , of course .\n \
              5,14	Just before Myra left -- Sue was saying good-by to Cathy , and she didm't realize I was near '' .\n \
              0	OOviously the farm should be on an all-weather road .") as f:
    for (locations, spellchk_sent) in spellchk(f):
        print("{locs}\t{sent}".format(
            locs=",".join([str(i) for i in locations]),
            sent=" ".join(spellchk_sent)
        ))

4	it will put your mind into non-stop learning.
3	`` Sad '' was not the right word , of course .
5,14	Just before Myra left -- She was saying good-by to Cathy , and she did realize I was near '' .
0	Normally the farm should be on an all-weather road .


Now, "Sue" is replaced with "She", which is completely correct. 
After checking different test cases, we noticed that the low accuracy of the approach stems from the language model. In other words, the correct word does not appear in the candidate list. For instance, in the above example, "Obviously" was not among the first 20 candidates. So, we thought maybe the language model is doing its job and we might just need to increase the number of candidates to capture the correct word. Therefore, we increased the number of candidates to 1000, which improved the accuracy significantly and made it 0.76.

In [11]:
def select_correction(typo, predict):
    # return the most likely prediction for the mask token
    LM_score = np.array([predict[i]['score'] for i in range(len(predict))])
    edit_dist = np.array([Levenshtein.distance(typo.lower(), predict[i]['token_str']) for i in range(len(predict))])
    edit_score = (edit_dist.max() - edit_dist) / edit_dist.max()
    w = 0.95
    score_tot = w * edit_score + (1-w) * LM_score
    best_ind = np.argmax(score_tot)
    return predict[best_ind]['token_str'].capitalize() if typo[0].isupper() else predict[best_ind]['token_str']
def spellchk(fh):
    for (locations, sent) in get_typo_locations(fh):
        spellchk_sent = sent
        for i in locations:
            # predict top_k replacements only for the typo word at index i
            predict = fill_mask(
                " ".join([ sent[j] if j != i else mask for j in range(len(sent)) ]), 
                top_k=1000
            )
            logging.info(predict)
            spellchk_sent[i] = select_correction(sent[i], predict)
        yield(locations, spellchk_sent)

In [13]:
with StringIO("0	OOviously the farm should be on an all-weather road .") as f:
    for (locations, spellchk_sent) in spellchk(f):
        print("{locs}\t{sent}".format(
            locs=",".join([str(i) for i in locations]),
            sent=" ".join(spellchk_sent)
        ))

0	Obviously the farm should be on an all-weather road .


Now, "Obviously" is also captured correctly. 

This was our final version of hw1 submitted to CourSys.

## Group work

* baa27 did all the work together with his groupmate mirzaei.
* mirzaei did all the work together with his groupmate baa27.
